In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe
mp_hands = mp.solutions.hands
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands=1
)
face_mesh = mp_face_mesh.FaceMesh(
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

# Load images
original_img = cv2.imread(r"C:\Users\farah\Downloads\original.jpg")
thumbs_up_img = cv2.imread(r"C:\Users\farah\Downloads\thumbs_up.jpg")
pointing_img = cv2.imread(r"C:\Users\farah\Downloads\pointing.jpg")
annoyed_img = cv2.imread(r"C:\Users\farah\Downloads\annoyed.jpg")
peace_img = cv2.imread(r"C:\Users\farah\Downloads\peace.jpg")
tongue_img = cv2.imread(r"C:\Users\farah\Downloads\download.jpg")
closed_eye_img = cv2.imread(r"C:\Users\farah\Downloads\closed eye.jpg")

if original_img is None or thumbs_up_img is None or pointing_img is None or annoyed_img is None or peace_img is None or tongue_img is None or closed_eye_img is None:
    print("Error: one or more images not found")
    print("Required images: original.jpg, thumbs_up.jpg, pointing.jpg, annoyed.jpg, peace.jpg, download.jpg, closed eye.jpg")
    exit()

# Get dimensions from original image
height, width = original_img.shape[:2]

# Resize all images to match
thumbs_up_img = cv2.resize(thumbs_up_img, (width, height))
pointing_img = cv2.resize(pointing_img, (width, height))
annoyed_img = cv2.resize(annoyed_img, (width, height))
peace_img = cv2.resize(peace_img, (width, height))
tongue_img = cv2.resize(tongue_img, (width, height))

# Start webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

current_img = original_img.copy()

def count_fingers(hand_landmarks):
    """Count extended fingers"""
    finger_count = 0
    
    # Finger tip indices: thumb, index, middle, ring, pinky
    tips = [4, 8, 12, 16, 20]
    pips = [3, 6, 10, 14, 18]
    
    # Check each finger
    for tip, pip in zip(tips, pips):
        tip_y = hand_landmarks.landmark[tip].y
        pip_y = hand_landmarks.landmark[pip].y
        
        # Finger is extended if tip is above pip
        if tip_y < pip_y:
            finger_count += 1
    
    return finger_count

def detect_thumbs_up(hand_landmarks):
    """Detect thumbs up gesture"""
    # Thumb tip should be above thumb base
    thumb_up = hand_landmarks.landmark[4].y < hand_landmarks.landmark[3].y
    
    # All other fingers should be down (curled)
    index_down = hand_landmarks.landmark[8].y > hand_landmarks.landmark[6].y
    middle_down = hand_landmarks.landmark[12].y > hand_landmarks.landmark[10].y
    ring_down = hand_landmarks.landmark[16].y > hand_landmarks.landmark[14].y
    pinky_down = hand_landmarks.landmark[20].y > hand_landmarks.landmark[18].y
    
    return thumb_up and index_down and middle_down and ring_down and pinky_down

def detect_pointing(hand_landmarks):
    """Detect pointing gesture (only index finger up)"""
    # Index finger should be extended
    index_up = hand_landmarks.landmark[8].y < hand_landmarks.landmark[6].y
    
    # All other fingers should be down
    thumb_down = hand_landmarks.landmark[4].y > hand_landmarks.landmark[3].y
    middle_down = hand_landmarks.landmark[12].y > hand_landmarks.landmark[10].y
    ring_down = hand_landmarks.landmark[16].y > hand_landmarks.landmark[14].y
    pinky_down = hand_landmarks.landmark[20].y > hand_landmarks.landmark[18].y
    
    return index_up and thumb_down and middle_down and ring_down and pinky_down

def detect_peace_sign(hand_landmarks):
    """Detect peace sign (index and middle fingers up)"""
    # Index and middle fingers should be extended
    index_up = hand_landmarks.landmark[8].y < hand_landmarks.landmark[6].y
    middle_up = hand_landmarks.landmark[12].y < hand_landmarks.landmark[10].y
    
    # Ring and pinky should be down
    ring_down = hand_landmarks.landmark[16].y > hand_landmarks.landmark[14].y
    pinky_down = hand_landmarks.landmark[20].y > hand_landmarks.landmark[18].y
    
    return index_up and middle_up and ring_down and pinky_down

def detect_mouth_open(face_landmarks):
    """Detect if mouth is open wide (tongue out)"""
    # Upper lip landmark: 13
    # Lower lip landmark: 14
    upper_lip = face_landmarks.landmark[13]
    lower_lip = face_landmarks.landmark[14]
    
    # Calculate distance between upper and lower lip
    mouth_distance = abs(upper_lip.y - lower_lip.y)
    
    # If mouth is open wide (threshold 0.03)
    return mouth_distance > 0.03

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Flip frame for mirror effect
    frame = cv2.flip(frame, 1)
    
    # Convert to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process frame for hands
    hand_results = hands.process(rgb_frame)
    
    # Process frame for face
    face_results = face_mesh.process(rgb_frame)
    
    finger_count = 0
    gesture_name = "None"
    mouth_open = False
    
    # Check for tongue out first (highest priority)
    if face_results.multi_face_landmarks:
        for face_landmarks in face_results.multi_face_landmarks:
            if detect_mouth_open(face_landmarks):
                current_img = tongue_img.copy()
                gesture_name = "Tongue Out"
                mouth_open = True
                break
    
    # Check hand gestures if mouth is not open
    if not mouth_open and hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            # Draw hand landmarks on frame
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS
            )
            
            # Count fingers
            finger_count = count_fingers(hand_landmarks)
            
            # Check for thumbs up
            if detect_thumbs_up(hand_landmarks):
                current_img = thumbs_up_img.copy()
                gesture_name = "Thumbs Up"
            # Check for peace sign
            elif detect_peace_sign(hand_landmarks):
                current_img = peace_img.copy()
                gesture_name = "Peace Sign"
            # Check for pointing
            elif detect_pointing(hand_landmarks):
                current_img = pointing_img.copy()
                gesture_name = "Pointing"
            # Check for open hand (5 fingers)
            elif finger_count >= 5:
                current_img = annoyed_img.copy()
                gesture_name = "Annoyed"
            else:
                current_img = original_img.copy()
                gesture_name = "Original"
    elif not mouth_open:
        current_img = original_img.copy()
        gesture_name = "Original"
    
    # Overlay camera frame on image
    frame_small = cv2.resize(frame, (width // 3, height // 3))
    x_offset = width - frame_small.shape[1] - 10
    y_offset = 10
    current_img[y_offset:y_offset + frame_small.shape[0],
                x_offset:x_offset + frame_small.shape[1]] = frame_small
    
    # Display info
    cv2.putText(current_img, f"Gesture: {gesture_name}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(current_img, f"Fingers: {finger_count}", (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # Show result
    cv2.imshow("Monster Control", current_img)
    
    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
hands.close()
face_mesh.close()

KeyboardInterrupt: 